# SWEN90006 Tutorial 8

## Introduction

The aim of this tutorial is for you to familiarise yourself with code coverage-guided fuzzing tools like AFL. In the class exercise last week (Week 8), we used generation-based/model-based black-box fuzzing tools like Peach fuzzer to generate inputs to trigger the faults in two versions of the read_and_process program. In this tutorial, you will use AFL to do the same task. 

## Working with programs

Following the instructions at https://github.com/SWEN90006-2021/security-testing to setup a Docker image and Docker container.

### First program: read_and_process_v1.c

To trigger the fault, called FAULT-1, in this simpler version of read_and_process, AFL needs to generate an input file that satisfies the following conditions:
-	The file should adhere to the structure of the expected file format. Recall that a valid file starts with a 4-byte "signature", followed by a list of chunks and each chunk has three parts: i) a chunk type stored in 4 bytes, ii) a 4-byte data length, and iii) the chunk data.
-	The list of chunks should have at least one data chunk of the "BOOM" type.

### Your tasks

Compile read_and_process_v1.c with afl-clang-fast so that AFL can be used to fuzz test the generated binary with code coverage feedback enabled.

```bash
afl-clang-fast -g -o read_and_process_v1 read_and_process_v1.c
```
Run AFL to fuzz test the program in four settings. If your computer has enough CPU cores (>= 4 cores), you can start four docker containers and run four settings at the same time to speed up the experiments. You can stop AFL by typing down the combination "Ctrl + C". You can also set up a timeout for each command if you wish by using the timeout command (see https://linuxize.com/post/timeout-command-in-linux/).

#### Setting-1
Use only a random file, containing a number like "1234", as seed input.

```bash
mkdir corpus-random
echo "1234" > corpus-random/random_file
afl-fuzz -d -i corpus-random -o out-setting-1 ./read_and_process_v1 @@
```

If you want to run afl-fuzz fuzzer for 5 minutes, you can use this command

```bash
timeout 5m afl-fuzz -d -i corpus-random -o out-setting-1 ./read_and_process_v1 @@
```
#### Setting-2
Use the same seed corpus in Setting-1 and use a so-called fuzzing dictionary. A fuzzing dictionary contains predefined tokens that can be randomly inserted into the generated input.

Create a file named fuzz.dict that contains three tokens as below. Inside the Docker container, you can use simple text editors like vim or nano to create this file.

```
"ABCD"
"BOOM"
"CHUK"
```

And run the following fuzzing command. The "-x" option is used to specify a fuzzing dictionary.

```bash
afl-fuzz -d -x fuzz.dict -i corpus-random -o out-setting-2 ./read_and_process_v1 @@
```

#### Setting-3
Use a seed corpus containing a valid input file that adheres to the specified format. The "printf" command is used to generate a valid file that has a single chunk of type "CHUK" and the chunk's data consists of four bytes. Note that all numbers are stored in little-endian order so "\x04\x00\x00\x00" is 0x04 in hexadecimal and 4 in decimal.

```bash
mkdir corpus-valid
printf "ABCDCHUK\x04\x00\x00\x00\x00\x00\x00\x00" > corpus-valid/valid_file
afl-fuzz -d -i corpus-valid -o out-setting-3 ./read_and_process_v1 @@
```

#### Setting-4
Use the same seed corpus in Setting-3 together with the fuzzing dictionary in Setting-2.

```bash
afl-fuzz -d -x fuzz.dict -i corpus-valid -o out-setting-4 ./read_and_process_v1 @@
```

Discuss the results in the four settings with a focus on the pros and cons of each setting. The crash-triggering inputs should be found in out-setting-*/crashes folder where normal inputs should be found in out-setting-*/queue folder.

### Second program: read_and_process_v2.c

To trigger the fault in this version of read_and_process, in addition to FAULT-1's conditions, AFL needs to generate an input file that satisfies the following conditions:
-	The data of the BOOM chunk contains two 4-byte integer numbers named x and y.
-	The values of x and y must satisfy this constraint: (x + y > 283) && (x + y < 286)

### Your tasks:

Compile read_and_process_v2.c with afl-clang-fast.

```bash
afl-clang-fast -g -o read_and_process_v2 read_and_process_v2.c
```

#### Setting-5
Use a seed corpus containing a valid input file that adheres to the specified format.

```bash
afl-fuzz -d -i corpus-valid -o out-setting-5 ./read_and_process_v2 @@
```

#### Setting-6
Use the same seed corpus in Setting-5 together with a fuzzing dictionary.

```bash
afl-fuzz -d -x fuzz.dict -i corpus-valid -o out-setting-6 ./read_and_process_v2 @@
```

Discuss the results in the two settings with a focus on the pros and cons of each setting. The crash-triggering inputs should be found in out-setting-*/crashes folder where normal inputs should be found in out-setting-*/queue folder.